
*Last update: August 2023*  
*Contact:* fdrs@ifrc.org or simon.weiss@ifrc.org  
*FDRS focal point:* Simon Weiss, FDRS Data Analyst


# How to deal with Missing Values? FDRS imputation methodology in python
-----------


The FDRS is ambitious and wide reaching. Although the data quality and reporting are improving each year, data is missing for some National Societies. As a result, some data fluctuations may be misleading: trend lines can drop for a given year when there is missing data, and some National Societies are excluded from the total and then appear again in another year.In order to better represent the network and better count everyone, FDRS implement every year data imputation techniques. 

The purpose of this notebook is to apply the method selected by the FDRS team and to interact with the FDRS backoffice with a post method to publish the imputed values. 
The ingested data is  replicated and displayed on the website https://data.ifrc.org/FDRS/ and used in FDRS research such as [Everyone count report](https://data-api.ifrc.org/documents/noiso/Everyone%20Counts%20Report%202022%20EN.pdf). 

Therefore this is not a research notebook but a production notebook. It aims to simply expose the FDRS methodology of imputation step by step. A related python script has been created: [imputing.py]() which allows to launch this methodology once. 

The approach chosen was to replace the 2019, 2020 and 2021 missing data as well as to apply two different techniques according to the indicator categories, in the previous years all NSs reported their data then no input technique was employed.   

The imputing applies only to main indicators and does not apply to disaggregated levels to maintain consistency across years. A detailed description of the methodology is available in the pdf [Missing Data]()


## Table of Contents


1. [Import Necessary Packages](#import-packages)
2. [Selecting Production and Staging Environment](#environment-selection)
3. [Data Retrieval](#data-retrieval)
4. [Data Imputation](#data-imputation)
5. [POST Method to Update Back-end](#post-method)

 

## 1. Imports and Initial Configurations

We'll start by loading the necessary Python modules and libraries for our analysis.


In [ ]:
import os
import sys
import time
import warnings
from datetime import datetime
import pandas as pd
import numpy as np
import requests
from pandas import json_normalize
from tqdm import tqdm
import json
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')

## 2. Configuration Loading

Before proceeding, let's select the desired environment. This will determine the configuration settings we'll use throughout the notebook.


In [ ]:
def select_environment(config_prod_path, config_staging_path):
    """Select the environment and load the respective configuration."""
    env = ''
    while env not in ['staging', 'prod']:
        env = input("Select environment ('staging' or 'prod'): ").lower()
        if env == 'prod':
            confirm = input("WARNING: You've selected the PRODUCTION environment. Are you sure? (yes/no) ")
            if confirm.lower() != 'yes':
                print("Switching to 'staging' by default for safety.")
                env = 'staging'
    try:
        with open(config_prod_path if env == 'prod' else config_staging_path, 'r') as file:
            return json.load(file)
    except FileNotFoundError:
        raise Exception(f"Configuration file for {env} not found.")

Configuration

In [ ]:
config_prod_path = '../src/config/config_prod.json'
config_staging_path = '../src/config/config_staging.json'
config = select_environment(config_prod_path, config_staging_path)
print(f"Loaded configuration for {config['ENV']} environment.")
user_email = input('Please provide your FDRS data analyst email: ')

## 3. Data Retrieval


### 3.1 Import necessary modules for data retrieval

In [ ]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(os.path.join(module_path, "src", "data"))
from fdrsapi import api_function, baseline, api_function_imputed


In [ ]:
#Generate fdrs codebook and store it in references folder

base_url = config['BASE_URL']
KPI = json_normalize(data=requests.get(f"{base_url}indicator?apiKey={config['API_KEY_PUBLIC']}").json())
codebook_path = os.path.join('..', 'references', 'codebook.xlsx')
KPI.to_excel(codebook_path)
codebook = pd.read_excel(codebook_path)

In [ ]:
codebook = pd.read_excel(r"..\references\codebook.xlsx")
kpi_reach = codebook.query('KPI_Note == "NS Reach - CPD"')["KPI_Code"].to_list() + ["KPI_TrainFA_Tot", "KPI_DonBlood_Tot"]
metadata_kpis = codebook.query('KPI_Note == "Metadata"')["KPI_Code"].tolist()
kpi_gov = ["KPI_GB_Tot", "KPI_PeopleVol_Tot", "KPI_PStaff_Tot", "KPI_noLocalUnits"]
kpi_fi = ["KPI_IncomeLC_CHF", "KPI_expenditureLC_CHF"]
kpi_gov_fi_code = kpi_gov + kpi_fi

exclude_kpis = ["KPI_Year", "DON_Code", "KPI_Id"]
metadata_kpis = [kpi for kpi in metadata_kpis if kpi not in exclude_kpis]
kpi_code = kpi_reach + kpi_gov_fi_code + metadata_kpis


In [ ]:
#counting nb of considered KPIs
print("number of total KPI considered:", len(kpi_code)-2)

### 3.2 Get data from FDRS API

In [ ]:
years = ["2018", "2019", "2020", "2021", "2022"]
time_series = api_function(years, kpi_code, config)
original_data = time_series.copy()
original_data

### 3.3 Analyzing FDRS data

In [ ]:
# Initialize a DataFrame to store the counts
metadata_counts = pd.DataFrame()

# Loop over each metadata KPI
for kpi in metadata_kpis:
    if "Date" in kpi:  # Check if the KPI is a date indicator
        yes_count = time_series[time_series[kpi].notna()].groupby('KPI_Year').size()
    else:
        yes_count = time_series[time_series[kpi] == 1.0].groupby('KPI_Year').size()

    no_count = time_series[time_series[kpi] == 0.0].groupby('KPI_Year').size()
    na_count = time_series[time_series[kpi].isna()].groupby('KPI_Year').size()
    
    # Construct a temporary DataFrame and append it to the main DataFrame
    temp_df = pd.DataFrame({
        'KPI': kpi,
        'True': yes_count,
        'False': no_count,
        'NA': na_count
    }).reset_index()
    
    metadata_counts = metadata_counts._append(temp_df, ignore_index=True)

# Pivot the dataframe for a better view
pivot_metadata_counts = metadata_counts.pivot(index='KPI', columns='KPI_Year')

# Display the metadata counts table
# Specify the order of KPIs to display at the top
priority_kpis = ['KPI_WasSubmitted', 'validated']

# Reorder the index to have priority KPIs at the top
pivot_metadata_counts = pivot_metadata_counts.reindex(priority_kpis + [kpi for kpi in pivot_metadata_counts.index if kpi not in priority_kpis])

# Display the pivoted table for comparison
pivot_metadata_counts


In [ ]:
time_series['KPI_FirstSubmitDate'] = pd.to_datetime(time_series['KPI_FirstSubmitDate'], errors='coerce')

# Extract month
time_series['Month'] = time_series['KPI_FirstSubmitDate'].dt.month

# Group by year and month, then compute the cumulative count
grouped_cumulative = time_series.groupby(['KPI_Year', 'Month']).size().groupby(level=0).cumsum().reset_index(name='Cumulative Count')

# Plot
plt.figure(figsize=(14, 7))
for year in time_series['KPI_Year'].unique():
    subset = grouped_cumulative[grouped_cumulative['KPI_Year'] == year]
    plt.plot(subset['Month'], subset['Cumulative Count'], label=year, marker='o')

plt.title('Cumulative Number of KPI_FirstSubmitDate Submissions Over the Year')
plt.xlabel('Month')
plt.ylabel('Cumulative Number of Submissions')
plt.xticks(np.arange(1, 13), ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
plt.legend()
plt.tight_layout()
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.show()


In [ ]:
time_series['KPI_NSR_SubmitDate'] = pd.to_datetime(time_series['KPI_NSR_SubmitDate'], errors='coerce')


# Extract month
time_series['Month'] = time_series['KPI_NSR_SubmitDate'].dt.month

# Group by year and month, then compute the cumulative count
grouped_cumulative = time_series.groupby(['KPI_Year', 'Month']).size().groupby(level=0).cumsum().reset_index(name='Cumulative Count')

# Plot
plt.figure(figsize=(14, 7))
for year in time_series['KPI_Year'].unique():
    subset = grouped_cumulative[grouped_cumulative['KPI_Year'] == year]
    plt.plot(subset['Month'], subset['Cumulative Count'], label=year, marker='o')

plt.title('Cumulative Number of NS Reach Sections Submissions Over the Year')
plt.xlabel('Month')
plt.ylabel('Cumulative Number of Submissions')
plt.xticks(np.arange(1, 13), ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
plt.legend()
plt.tight_layout()
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.show()


# 4. Data Imputation

In [ ]:
def impute_data(df, kpi_reach_code, kpi_gov_fi_code, selected_year):
    """
    Impute missing data in the dataframe based on various conditions.
    
    Parameters:
        df (pd.DataFrame): The original dataframe with potential missing values.
        kpi_reach_code (list): List of KPI codes related to reach.
        kpi_gov_fi_code (list): List of KPI codes related to governance and financial indicators.
        selected_year (int): The year for which to perform the imputation.
        
    Returns:
        pd.DataFrame: Dataframe with imputed values.
    """

    data = df.copy()
    condition_selected_year = data['KPI_Year'] == float(selected_year)
    condition_not_submitted = data['KPI_WasSubmitted'] != 1.0
    condition_nsgs_not_submitted = data['KPI_NSGS_WasSubmitted'] != 1.0

    for col in kpi_reach_code:
        mask = condition_selected_year & condition_not_submitted & data[col].isna()
        if mask.any():
            data[col] = data.groupby("NSO_DON_name")[col].transform(lambda x: x.fillna(x.mean(), limit=3))

    for col in kpi_gov_fi_code:
        mask = condition_selected_year & condition_not_submitted & data[col].isna()
        if mask.any():
            data[col] = data.groupby("NSO_DON_name")[col].transform(lambda x: x.fillna(method='ffill', limit=3))

    condition_special = (data['KPI_WasSubmitted'] == 1.0) & condition_nsgs_not_submitted
    for col in kpi_gov:
        mask = condition_selected_year & condition_special & data[col].isna()
        if mask.any():
            data[col] = data.groupby("NSO_DON_name")[col].transform(lambda x: x.fillna(method='ffill', limit=3))

    return data

In [ ]:
# Perform imputation for the year 2022
imputed_data_2022 = impute_data(time_series, kpi_reach, kpi_gov_fi_code, 2022)

### Imputing Analysis

In [ ]:
imputed_data_2021 = impute_data(time_series, kpi_reach, kpi_gov_fi_code, 2021)
imputed_data_2022 = impute_data(time_series, kpi_reach, kpi_gov_fi_code, 2022)

# Compare imputed and original (non-imputed) data
print("\nComparison (Original vs. Imputed) for 2022:")
print("Original:", time_series[time_series["KPI_Year"] == 2022][["NSO_DON_name", "KPI_PeopleVol_Tot"]].sum())
print("Imputed:", imputed_data_2022[imputed_data_2022["KPI_Year"] == 2022][["NSO_DON_name", "KPI_PeopleVol_Tot"]].sum())

print("\nComparison (Original vs. Imputed) for 2021:")
print("Original:", time_series[time_series["KPI_Year"] == 2021][["NSO_DON_name", "KPI_PeopleVol_Tot"]].sum())
print("Imputed:", imputed_data_2021[imputed_data_2021["KPI_Year"] == 2021][["NSO_DON_name", "KPI_PeopleVol_Tot"]].sum())

# Compare imputed and original (non-imputed) data
print("\nComparison (Original vs. Imputed) for 2022:")
print("Original:", time_series[time_series["KPI_Year"] == 2022][["NSO_DON_name", "KPI_IncomeLC_CHF"]].sum())
print("Imputed:", imputed_data_2022[imputed_data_2022["KPI_Year"] == 2022][["NSO_DON_name", "KPI_IncomeLC_CHF"]].sum())

print("\nComparison (Original vs. Imputed) for 2021:")
print("Original:", time_series[time_series["KPI_Year"] == 2021][["NSO_DON_name", "KPI_IncomeLC_CHF"]].sum())
print("Imputed:", imputed_data_2021[imputed_data_2021["KPI_Year"] == 2021][["NSO_DON_name", "KPI_IncomeLC_CHF"]].sum())


# Compare imputed and original (non-imputed) data for KPI_ReachDRER_CPD_IP
print("\nComparison (Original vs. Imputed) for KPI_ReachDRER_CPD in 2022:")
print("Original:", time_series[time_series["KPI_Year"] == 2022][["NSO_DON_name", "KPI_ReachDRER_CPD"]].sum())
print("Imputed:", imputed_data_2022[imputed_data_2022["KPI_Year"] == 2022][["NSO_DON_name", "KPI_ReachDRER_CPD"]].sum())


print("\nComparison (Original vs. Imputed) for KPI_ReachDRER_CPD in 2021:")
print("Original:", time_series[time_series["KPI_Year"] == 2021][["NSO_DON_name", "KPI_ReachDRER_CPD"]].sum())
print("Imputed:", imputed_data_2021[imputed_data_2021["KPI_Year"] == 2021][["NSO_DON_name", "KPI_ReachDRER_CPD"]].sum())

In [ ]:
#filter to NSO DON Name DIN001 and 2022 for PeopleVol

imputed_data_2022[(imputed_data_2022["KPI_Year"] == 2022) & (imputed_data_2022["NSO_DON_name"] == "Indian Red Cross Society")][["NSO_DON_name", "KPI_PeopleVol_Tot"]]

## 5. POST Method to Update Back-end
---------------

In [ ]:
def prepare_data_for_post(original_dataframe, imputed_dataframe, kpi_list, selected_year):
    # Filter dataframes by the selected year
    original_subset = original_dataframe[original_dataframe["KPI_Year"] == selected_year].copy()
    imputed_subset = imputed_dataframe[imputed_dataframe["KPI_Year"] == selected_year].copy()

    # For each KPI, iterate through each row in the imputed dataframe
    for kpi in kpi_list:
        for index, row in imputed_subset.iterrows():
            don_code = row['KPI_DON_code']
            original_value = original_subset[(original_subset['KPI_DON_code'] == don_code)][kpi].values[0]
            imputed_value = row[kpi]

            # Determine the source value based on the comparison
            source = 'NSI' if original_value == imputed_value else 'I'
            imputed_subset.at[index, f"{kpi}_source"] = source

    return imputed_subset

In [ ]:
imputed_data_prepared = prepare_data_for_post(original_data, imputed_data_2022, kpi_reach + kpi_gov_fi_code, 2022)

In [ ]:
def post_imputed_data(dataframe, kpi_list, selected_year):
    total_kpis = len(kpi_list)
    total_dons = len(set(dataframe["KPI_DON_code"]))
    total_requests = total_kpis * total_dons

    sleep_time = 0.5
    estimated_time = total_requests * sleep_time / 60
    print(f"Estimated time for completion: {estimated_time:.2f} minutes.")
    
    failed_posts = []  # To keep track of any failed POST requests

    with tqdm(total=total_requests, desc="Posting Data", unit="Request") as pbar:
        for kpi in kpi_list:
            kpi_ip = kpi + "_IP"
            source_column_name = kpi + "_source"
            for kpi_don_code in set(dataframe["KPI_DON_code"]):
                value_row = dataframe[(dataframe["KPI_Year"] == selected_year) & (dataframe["KPI_DON_code"] == kpi_don_code)]
                
                value = value_row[kpi].values[0]
                source_value = value_row[source_column_name].values[0]

                if pd.isna(value):
                    value = ""
                elif isinstance(value, float):
                    value = int(value)

                url = f"{config['BASE_URL']}ImputedKPI?apiKey={config['API_KEY_PRIVATE']}&kpicode={kpi_ip}&year={selected_year}&don_code={kpi_don_code}&value={value}&source={source_value}&user={user_email}"
                
                response = requests.post(url)
                time.sleep(sleep_time)

                # Check response status code
                if response.status_code != 200:
                    print(f"Failed to post data for KPI: {kpi}, DON: {kpi_don_code}. Server responded with {response.status_code}: {response.text}")
                    failed_posts.append((kpi, kpi_don_code, response.status_code, response.text))
                
                pbar.update(1)

    # Once done, if there were any failures, print a summary
    if failed_posts:
        print("\nSummary of failed POSTs:")
        for kpi, don, status_code, message in failed_posts:
            print(f"KPI: {kpi}, DON: {don} failed with {status_code}: {message}")

In [ ]:
post_imputed_data(imputed_data_prepared, kpi_reach + kpi_gov_fi_code, 2022)
